# Set up

In [ ]:
!pip install pycaret

In [ ]:
!pip install tslearn

In [ ]:
import numpy as np; np.random.seed(42)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from pycaret import classification
from pycaret import regression
import pycaret
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir 'Data'
!unzip 'drive/MyDrive/Kaggle/electric-power-consumption.zip' -d 'Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘Data’: File exists
Archive:  drive/MyDrive/Kaggle/electric-power-consumption.zip
  inflating: Data/PowerCons_TEST.csv  
  inflating: Data/PowerCons_TRAIN.csv  
  inflating: Data/README.md          
  inflating: Data/forecast_submission_example.csv  


# Prepare data and predict

In [ ]:
train = pd.read_csv('/content/Data/PowerCons_TRAIN.csv')
test = pd.read_csv('/content/Data/PowerCons_TEST.csv')

ตอนนี้พยายามแปลงข้อมูล train set ให้เหมือน test set 

In [ ]:
half_train=train.iloc[:,1:73]
another_half_train=train.iloc[:,73:len(train.columns)-2]
half_train['kW-12:00-23:50']=another_half_train.sum(axis=1)
half_train['Class']=train['Class'].astype(int)
half_train=half_train.rename(columns={'kW-12:00-23:50':'label'})
half_train['Class']=half_train['Class'].map({1:'Hot',2:'Cold'})
half_train.head()

,00:00,00:10,00:20,00:30,00:40,00:50,01:00,01:10,01:20,01:30,01:40,01:50,02:00,02:10,02:20,02:30,02:40,02:50,03:00,03:10,03:20,03:30,03:40,03:50,04:00,04:10,04:20,04:30,04:40,04:50,05:00,05:10,05:20,05:30,05:40,05:50,06:00,06:10,06:20,06:30,06:40,06:50,07:00,07:10,07:20,07:30,07:40,07:50,08:00,08:10,08:20,08:30,08:40,08:50,09:00,09:10,09:20,09:30,09:40,09:50,10:00,10:10,10:20,10:30,10:40,10:50,11:00,11:10,11:20,11:30,11:40,11:50,label,Class
0,0.44,0.40,0.36,0.35,0.35,0.38,0.36,0.35,0.35,0.38,0.38,0.38,0.42,0.45,0.43,0.37,0.66,1.13,1.35,1.09,0.75,0.41,0.37,0.33,0.33,0.40,0.40,0.39,0.40,0.43,0.40,0.35,0.31,0.33,0.33,0.33,0.30,0.30,0.33,0.36,1.10,1.83,1.93,2.04,2.04,1.93,1.93,1.53,1.36,1.19,2.25,3.31,3.08,2.85,2.04,1.24,1.26,1.29,1.31,1.34,1.79,2.25,1.73,1.21,1.12,0.97,0.82,0.47,0.11,0.16,0.22,0.62,41.30,Hot
1,0.14,0.19,0.19,0.16,0.16,0.18,0.17,0.16,0.17,0.17,0.16,0.18,0.18,0.14,0.13,0.16,0.19,0.14,0.15,0.18,0.18,0.15,0.16,0.19,0.17,0.16,0.16,0.17,0.16,0.18,0.17,0.14,0.15,0.17,0.18,0.10,0.37,0.64,0.89,1.14,1.19,1.24,0.68,0.12,0.16,0.17,0.16,0.18,0.16,0.14,0.15,0.16,0.16,0.13,0.16,0.14,0.12,0.14,0.17,0.18,0.16,0.16,0.16,0.16,0.17,0.22,0.25,0.25,0.27,0.29,0.17,0.13,27.79,Hot
2,0.34,0.34,0.34,0.26,0.18,0.83,1.48,1.36,0.95,0.55,0.42,0.29,0.26,0.24,0.24,0.25,0.23,0.25,0.26,0.25,0.25,0.26,0.28,0.30,0.33,0.31,0.30,0.27,0.25,0.22,0.20,0.22,0.24,0.22,0.19,0.22,0.24,0.24,0.25,1.79,3.33,2.94,2.54,2.54,1.58,1.58,1.46,1.34,1.61,1.79,1.96,1.97,1.63,1.29,1.31,1.34,1.32,1.30,1.28,1.26,1.24,1.21,0.92,0.64,0.45,0.26,0.23,0.24,0.26,0.25,0.25,0.29,51.04,Hot
3,1.70,1.50,1.30,3.40,5.50,4.15,2.80,2.67,2.53,2.49,2.45,2.41,1.64,0.88,0.53,0.19,0.17,0.16,0.16,0.22,0.29,0.19,0.14,0.15,0.21,0.26,0.26,0.22,0.21,0.24,0.26,0.29,0.22,0.17,0.13,0.19,0.24,0.16,0.13,0.17,0.21,0.26,0.30,0.99,1.67,1.52,1.36,1.38,1.38,1.39,1.41,1.38,1.35,1.34,1.33,1.33,1.31,1.30,0.89,0.49,0.31,0.14,0.20,0.23,0.22,0.23,0.28,0.31,0.30,0.28,0.34,0.41,67.77,Hot
4,1.78,1.59,1.39,1.45,1.45,1.41,1.40,1.45,1.46,1.40,0.85,0.30,0.34,0.38,0.33,0.33,0.39,0.35,0.31,0.31,0.35,0.38,0.38,0.33,0.33,0.38,0.39,0.35,0.33,0.36,0.38,0.37,0.34,0.34,0.37,0.37,0.35,0.35,0.37,0.38,0.36,0.35,0.36,0.36,0.36,0.35,0.37,0.37,0.35,0.34,0.36,0.38,0.37,0.39,0.41,1.08,1.75,1.68,1.66,1.65,1.50,1.34,1.55,1.75,1.68,1.60,1.48,1.41,1.38,1.33,1.31,1.34,31.31,Hot


สร้าง model สำหรับทำนาย class เนื่องจากข้อมูล test set ไม่มี class

In [ ]:
classification_model=classification.setup(data=half_train, target='Class', train_size=0.8,
               numeric_features=list(half_train.drop(['label','Class'],axis=1).columns),
               ignore_features=['label'],
               normalize=True,
               silent=True)

best_class_model=classification.compare_models(sort='Accuracy',turbo=False)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8176,0.8671,0.8482,0.8102,0.8205,0.6307,0.6454,0.522
et,Extra Trees Classifier,0.8176,0.8871,0.8482,0.8100,0.8242,0.6339,0.6440,0.472
lightgbm,Light Gradient Boosting Machine,0.8095,0.8165,0.8321,0.8044,0.8114,0.6149,0.6269,0.088
gbc,Gradient Boosting Classifier,0.7895,0.8619,0.8339,0.7827,0.8004,0.5736,0.5888,0.191
ada,Ada Boost Classifier,0.7681,0.7676,0.8089,0.7592,0.7786,0.5343,0.5459,0.130
mlp,MLP Classifier,0.7414,0.7937,0.7821,0.7542,0.7615,0.4779,0.4886,0.461
lr,Logistic Regression,0.7276,0.7956,0.7321,0.7547,0.7367,0.4501,0.4594,0.373
rbfsvm,SVM - Radial Kernel,0.7152,0.7761,0.7982,0.7087,0.7427,0.4196,0.4398,0.025
lda,Linear Discriminant Analysis,0.7071,0.7489,0.8089,0.7019,0.7434,0.4014,0.4250,0.020
ridge,Ridge Classifier,0.7067,0.0000,0.7304,0.7358,0.7213,0.4074,0.4284,0.016


In [ ]:
test_nodate=test.drop('Day',axis=1)
test_class=classification.predict_model(best_class_model, data=test_nodate)['Label'].map({'Hot':1,'Cold':2})
test_class

0      2
1      1
2      2
3      1
4      1
      ..
175    1
176    1
177    2
178    2
179    2
Name: Label, Length: 180, dtype: int64

เนื่องจากข้อมูลมีความ multicolinear สูงจึงทำการหา difference เผื่อมันจะช่วยแก้ปัญหานี้ได้ก่อนที่จะนำไปเทรน regression

In [ ]:
X=half_train.drop(['label'],axis=1)
y=half_train['label']

diff_train=pd.DataFrame()

first=list(X.columns)[:-2]
second=list(X.columns)[1:-1]
for t1,t2 in zip(first,second):
  diff_train=pd.concat([diff_train,X[t2]-X[t1]],axis=1)
diff_train.columns=['t'+str(i) for i in range(len(X.columns)-2)]
diff_train=pd.concat([diff_train,X['Class'],y],axis=1)
diff_train['Class']=diff_train['Class'].map({'Hot':1,'Cold':2})

diff_train

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,Class,label
0,-0.04,-0.04,-0.01,0.00,0.03,-0.02,-0.01,0.00,0.03,0.00,0.00,0.04,0.03,-0.02,-0.06,0.29,0.47,0.22,-0.26,-0.34,-0.34,-0.04,-0.04,0.00,0.07,0.00,-0.01,0.01,0.03,-0.03,-0.05,-0.04,0.02,0.00,0.00,-0.03,0.00,0.03,0.03,0.74,0.73,0.10,0.11,0.00,-0.11,0.00,-0.40,-0.17,-0.17,1.06,1.06,-0.23,-0.23,-0.81,-0.80,0.02,0.03,0.02,0.03,0.45,0.46,-0.52,-0.52,-0.09,-0.15,-0.15,-0.35,-0.36,0.05,0.06,0.40,1,41.30
1,0.05,0.00,-0.03,0.00,0.02,-0.01,-0.01,0.01,0.00,-0.01,0.02,0.00,-0.04,-0.01,0.03,0.03,-0.05,0.01,0.03,0.00,-0.03,0.01,0.03,-0.02,-0.01,0.00,0.01,-0.01,0.02,-0.01,-0.03,0.01,0.02,0.01,-0.08,0.27,0.27,0.25,0.25,0.05,0.05,-0.56,-0.56,0.04,0.01,-0.01,0.02,-0.02,-0.02,0.01,0.01,0.00,-0.03,0.03,-0.02,-0.02,0.02,0.03,0.01,-0.02,0.00,0.00,0.00,0.01,0.05,0.03,0.00,0.02,0.02,-0.12,-0.04,1,27.79
2,0.00,0.00,-0.08,-0.08,0.65,0.65,-0.12,-0.41,-0.40,-0.13,-0.13,-0.03,-0.02,0.00,0.01,-0.02,0.02,0.01,-0.01,0.00,0.01,0.02,0.02,0.03,-0.02,-0.01,-0.03,-0.02,-0.03,-0.02,0.02,0.02,-0.02,-0.03,0.03,0.02,0.00,0.01,1.54,1.54,-0.39,-0.40,0.00,-0.96,0.00,-0.12,-0.12,0.27,0.18,0.17,0.01,-0.34,-0.34,0.02,0.03,-0.02,-0.02,-0.02,-0.02,-0.02,-0.03,-0.29,-0.28,-0.19,-0.19,-0.03,0.01,0.02,-0.01,0.00,0.04,1,51.04
3,-0.20,-0.20,2.10,2.10,-1.35,-1.35,-0.13,-0.14,-0.04,-0.04,-0.04,-0.77,-0.76,-0.35,-0.34,-0.02,-0.01,0.00,0.06,0.07,-0.10,-0.05,0.01,0.06,0.05,0.00,-0.04,-0.01,0.03,0.02,0.03,-0.07,-0.05,-0.04,0.06,0.05,-0.08,-0.03,0.04,0.04,0.05,0.04,0.69,0.68,-0.15,-0.16,0.02,0.00,0.01,0.02,-0.03,-0.03,-0.01,-0.01,0.00,-0.02,-0.01,-0.41,-0.40,-0.18,-0.17,0.06,0.03,-0.01,0.01,0.05,0.03,-0.01,-0.02,0.06,0.07,1,67.77
4,-0.19,-0.20,0.06,0.00,-0.04,-0.01,0.05,0.01,-0.06,-0.55,-0.55,0.04,0.04,-0.05,0.00,0.06,-0.04,-0.04,0.00,0.04,0.03,0.00,-0.05,0.00,0.05,0.01,-0.04,-0.02,0.03,0.02,-0.01,-0.03,0.00,0.03,0.00,-0.02,0.00,0.02,0.01,-0.02,-0.01,0.01,0.00,0.00,-0.01,0.02,0.00,-0.02,-0.01,0.02,0.02,-0.01,0.02,0.02,0.67,0.67,-0.07,-0.02,-0.01,-0.15,-0.16,0.21,0.20,-0.07,-0.08,-0.12,-0.07,-0.03,-0.05,-0.02,0.03,1,31.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,-0.01,-0.04,-0.05,-0.02,-0.01,-0.02,0.01,0.01,0.56,0.55,-0.07,-0.14,-0.13,-0.37,-0.37,-0.01,-0.01,0.00,0.00,0.01,0.02,0.01,0.00,-0.03,-0.03,0.03,0.03,0.04,0.00,-0.05,0.00,0.01,-0.04,-0.03,0.05,-0.01,-0.01,0.01,0.02,0.93,0.93,-0.08,-0.08,0.01,0.00,0.05,0.05,0.07,0.06,0.31,0.31,-0.15,-0.15,-0.18,-0.19,-0.08,-0.09,-0.21,-0.10,-0.10,-0.04,-0.03,1.07,1.07,0.05,-0.46,-0.46,-0.59,-0.59,0.03,0.03,2,121.96
176,-0.22,-0.22,0.04,0.05,0.07,-0.01,-0.02,-0.09,-0.04,-0.05,0.02,0.02,0.01,0.03,0.04,-0.04,-0.05,0.03,-0.01,-0.02,0.00,0.02,-0.02,-0.02,0.00,0.48,0.49,0.06,0.05,-0.19,-0.44,-0.43,0.02,0.02,-0.05,-0.01,0.07,0.08,-0.09,1.13,1.13,0.03,-0.20,-0.19,0.24,0.25,-0.45,-0.45,0.42,0.41,-1.10,-1.10,-0.05,-0.06,0.11,0.34,0.34,-0.32,-0.33,0.02,0.02,-0.06,-0.07,0.02,0.02,0.08,0.05,0.04,0.15,0.14,-0.08,2,114.52
177,-0.08,-0.05,0.04,0.09,-0.01,-0.07,-0.05,0.03,0.03,-0.02,-0.04,0.01,0.04,0.01,-0.04,0.01,0.06,0.03,-0.05,-0.05,0.00,0.01,0.00,-0.02,0.01,0.01,0.01,-0.03,0.03,0.05,0.01,-0.05,-0.04,0.01,0.04,0.37,0.37,0.34,0.35,0.15,0.16,2.87,2.87,-2.25,-2.25,-0.60,-0.60,0.09,0.10,-0.44,-0.44,0.02,-0.02,0.61,0.62,-0.63,-0.63,-0.02,0.03,0.04,-0.02,-0.05,-0.04,0.04,0.03,-0.02,-0.04,0.01,0.05,0.01,-0.02,2,176.74
178,-0.07,-0.05,-0.06,0.04,0.05,-0.07,-0.01,0.04,0.04,0.00,-0.03,0.04,0.03,-0.05,-0.05,-0.02,0.03,0.00,0.00,0.01,0.03,0.02,-0.02,-0.05,-0.03,0.01,0.01,0.02,-0.02,0.00,0.06,0.06,-0.07,-0.08,0.01,0.04,0.04,-0.06,0.00,0.04,1

In [ ]:
# reg_model=regression.setup(data=diff_train, target='label', train_size=0.8,
#                numeric_features=list(diff_train.drop(['label'],axis=1).columns),
#                categorical_features=['Class'],
#                normalize=True,
#                silent=True)

# best_reg_model=regression.compare_models(sort='RMSE',turbo=False)

In [ ]:
hot_train=diff_train.loc[diff_train.Class==1,:].drop('Class',axis=1)
cold_train=diff_train.loc[diff_train.Class==2,:].drop('Class',axis=1)
print(hot_train.head())
print(cold_train.head())

     t0    t1    t2    t3    t4    t5  ...   t66   t67   t68   t69   t70  label
0 -0.04 -0.04 -0.01  0.00  0.03 -0.02  ... -0.35 -0.36  0.05  0.06  0.40  41.30
1  0.05  0.00 -0.03  0.00  0.02 -0.01  ...  0.00  0.02  0.02 -0.12 -0.04  27.79
2  0.00  0.00 -0.08 -0.08  0.65  0.65  ...  0.01  0.02 -0.01  0.00  0.04  51.04
3 -0.20 -0.20  2.10  2.10 -1.35 -1.35  ...  0.03 -0.01 -0.02  0.06  0.07  67.77
4 -0.19 -0.20  0.06  0.00 -0.04 -0.01  ... -0.07 -0.03 -0.05 -0.02  0.03  31.31

[5 rows x 72 columns]
      t0    t1    t2    t3    t4    t5  ...   t66   t67   t68   t69   t70   label
30 -0.02  0.00  0.02 -0.02 -0.03 -0.02  ...  0.05  0.05 -0.03  0.00  0.03   97.17
31  0.00 -1.10  0.00 -0.01 -0.02 -0.03  ...  0.01  0.05  0.03 -0.01  0.00   75.32
32  0.19  0.20  0.37  0.09  0.09 -0.01  ...  0.00 -0.38  0.00  0.08 -0.51  178.64
33  0.04  0.05  0.05 -0.02 -0.05 -0.04  ... -0.03  0.02  0.07 -0.02 -0.03   84.48
34  0.06  0.03 -0.06  0.01  0.06  0.01  ... -0.29 -0.28 -0.11 -0.03  0.03  135.57

[5 r

In [ ]:
hot_reg_model=pycaret.regression.setup(data=hot_train, target='label', train_size=0.8,
               numeric_features=list(hot_train.drop(['label'],axis=1).columns),
               normalize=True,
               silent=True)

best_hot_reg_model=pycaret.regression.compare_models(sort='RMSE',turbo=False)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.478340e+01,4.120568e+02,1.948310e+01,-3.540000e-02,0.3621,3.227000e-01,0.419
gbr,Gradient Boosting Regressor,1.444250e+01,4.212093e+02,1.948790e+01,-2.700000e-02,0.3622,3.125000e-01,0.096
rf,Random Forest Regressor,1.564250e+01,4.326019e+02,1.975940e+01,-2.140000e-02,0.3685,3.429000e-01,0.464
lightgbm,Light Gradient Boosting Machine,1.642990e+01,4.756330e+02,2.078350e+01,-1.975000e-01,0.3911,3.684000e-01,0.026
ada,AdaBoost Regressor,1.708920e+01,4.861673e+02,2.134650e+01,-2.103000e-01,0.4020,3.783000e-01,0.099
en,Elastic Net,1.775430e+01,4.969678e+02,2.138770e+01,-2.271000e-01,0.4001,3.847000e-01,0.015
svm,Support Vector Regression,1.743040e+01,4.991633e+02,2.159610e+01,-1.985000e-01,0.3937,3.672000e-01,0.017
dummy,Dummy Regressor,1.808090e+01,5.091497e+02,2.187880e+01,-2.638000e-01,0.4070,4.050000e-01,0.010
llar,Lasso Least Angle Regression,1.808530e+01,5.103986e+02,2.190210e+01,-2.657000e-01,0.4073,4.047000e-01,0.013
br,Bayesian Ridge,1.815350e+01,5.157759e+02,2.192870e+01,-2.559000e-01,0.4077,4.019000e-01,0.024


In [ ]:
cold_reg_model=pycaret.regression.setup(data=cold_train, target='label', train_size=0.8,
               numeric_features=list(cold_train.drop(['label'],axis=1).columns),
               normalize=True,
               silent=True)

best_cold_reg_model=pycaret.regression.compare_models(sort='RMSE',turbo=False)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,2.391140e+01,1.037327e+03,2.960080e+01,3.100000e-02,0.2216,1.940000e-01,0.466
et,Extra Trees Regressor,2.196340e+01,1.062935e+03,2.978010e+01,8.220000e-02,0.2245,1.784000e-01,0.409
gbr,Gradient Boosting Regressor,2.225500e+01,1.119584e+03,3.037770e+01,-1.969000e-01,0.2283,1.820000e-01,0.097
ada,AdaBoost Regressor,2.438830e+01,1.081035e+03,3.053930e+01,-1.992000e-01,0.2297,1.961000e-01,0.097
lightgbm,Light Gradient Boosting Machine,2.516480e+01,1.092127e+03,3.123600e+01,-6.858000e-01,0.2352,2.038000e-01,0.025
llar,Lasso Least Angle Regression,2.894990e+01,1.331041e+03,3.451250e+01,-1.850000e-01,0.2607,2.390000e-01,0.014
br,Bayesian Ridge,2.983320e+01,1.416491e+03,3.516180e+01,-2.285000e-01,0.2653,2.436000e-01,0.019
en,Elastic Net,3.023230e+01,1.484295e+03,3.615570e+01,-5.070000e-01,0.2733,2.478000e-01,0.015
dummy,Dummy Regressor,3.203340e+01,1.483475e+03,3.668410e+01,-3.377000e-01,0.2776,2.626000e-01,0.011
knn,K Neighbors Regressor,3.139770e+01,1.522629e+03,3.698110e+01,-8.807000e-01,0.2802,2.491000e-01,0.062


In [ ]:
diff_test=pd.DataFrame()

first=list(test_nodate.columns)[:-1]
second=list(test_nodate.columns)[1:]
for t1,t2 in zip(first,second):
  diff_test=pd.concat([diff_test,test_nodate[t2]-test_nodate[t1]],axis=1)
diff_test.columns=['t'+str(i) for i in range(len(test_nodate.columns)-1)]
diff_test=pd.concat([diff_test,test_class],axis=1)
diff_test=diff_test.rename(columns={'Label':'Class'})
diff_test

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,Class
0,-0.03,-0.01,-0.01,0.00,0.01,0.01,0.49,0.48,0.06,0.06,-0.57,-0.56,0.00,0.06,-0.01,-0.10,0.09,0.00,-0.01,-0.01,-0.01,-0.02,0.01,0.01,-0.06,0.08,0.07,-0.07,-0.08,0.03,0.03,-0.02,-0.02,0.01,0.01,0.03,0.02,-0.03,-0.04,0.78,0.78,-0.33,0.27,0.27,0.38,0.37,0.00,-1.23,0.00,-0.14,-0.03,-0.04,0.02,-0.03,-0.03,-0.01,0.00,0.04,0.03,-0.01,-0.01,-0.50,-0.50,0.00,-0.05,0.00,0.00,0.11,0.00,0.74,0.74,2
1,0.00,0.00,0.00,0.04,0.00,-0.04,-0.03,0.02,0.01,0.02,-0.01,-0.01,0.00,0.03,0.00,-0.04,-0.05,0.00,0.01,0.02,0.03,-0.03,0.02,0.02,-0.05,-0.05,0.00,0.00,0.06,0.06,-0.07,-0.07,0.00,0.06,0.00,0.02,0.02,0.00,0.00,1.20,1.20,-0.53,-0.52,0.02,0.02,-0.42,-0.42,0.65,0.66,-0.25,-0.25,0.09,0.01,0.02,-0.28,-0.10,-0.09,0.09,-0.05,-0.06,0.00,0.01,0.00,0.00,-0.21,-0.36,-0.35,0.00,-0.06,0.00,0.58,1
2,-0.28,-0.28,-0.64,-0.65,-0.05,0.00,0.01,0.00,-0.03,-0.02,0.02,0.01,0.00,-0.01,0.00,0.03,0.00,-0.04,-0.04,0.01,0.01,0.02,0.02,0.00,0.10,0.00,-0.07,-0.08,0.00,0.00,0.04,0.04,0.00,-0.03,0.00,0.02,0.01,0.26,0.26,0.77,0.53,0.52,0.02,0.02,0.04,0.03,0.02,0.08,0.09,-0.62,-0.63,0.41,-0.19,-0.19,-0.22,-0.23,0.09,0.03,0.03,0.00,-0.07,0.00,-0.01,-0.02,-0.04,0.00,0.11,0.00,0.08,0.09,0.32,2
3,-0.43,-0.36,-0.37,-0.65,-0.11,-0.11,-0.16,-0.28,-0.28,-0.38,-0.19,-0.20,0.02,0.01,0.00,0.01,0.02,0.00,0.00,0.00,0.01,0.01,0.02,-0.04,-0.03,0.02,0.01,-0.04,-0.04,0.01,0.03,0.03,-0.01,-0.01,0.03,-0.01,-0.01,-0.01,-0.01,0.02,0.01,-0.03,-0.04,0.23,0.22,0.28,0.29,0.00,0.06,0.00,0.00,0.85,0.85,-1.30,-1.30,1.00,0.99,-0.91,-0.92,0.42,0.43,-0.02,-0.01,1.28,1.28,-0.83,-0.83,0.50,0.49,-0.06,-0.05,1
4,-0.11,-0.10,-0.05,0.01,0.04,0.03,-0.01,-0.01,-0.02,-0.01,-0.01,-0.01,-0.04,0.55,0.56,-0.06,-0.41,-0.41,-0.13,-0.07,0.01,0.03,-0.02,-0.02,0.02,0.01,-0.01,0.01,-0.02,0.00,0.03,0.00,-0.05,0.01,0.04,-0.01,-0.03,0.01,0.02,0.00,0.48,0.48,-0.37,-0.37,0.23,0.22,0.23,0.22,-0.11,-0.10,-0.24,-0.24,-0.25,-0.24,0.03,0.01,0.00,0.01,-0.03,0.00,0.03,0.00,-0.04,0.01,0.03,0.00,-0.02,0.00,0.01,0.00,0.01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.00,-0.01,-0.03,-0.01,0.00,-0.03,-0.01,0.04,0.04,-0.02,0.01,0.01,-0.03,-0.04,0.02,0.02,0.03,-0.01,-0.01,-0.01,-0.02,0.00,0.00,0.01,0.03,0.03,-0.03,-0.03,-0.01,0.00,0.01,0.00,-0.03,-0.02,0.06,0.06,-0.06,-0.06,0.30,0.29,0.25,0.25,0.06,0.07,-0.59,-0.59,-0.01,0.00,0.04,0.03,-0.01,0.00,0.02,-0.05,-0.06,0.04,0.04,0.03,-0.01,-0.02,0.01,-0.01,-0.02,-0.02,-0.01,0.03,0.04,-0.02,-0.01,0.01,0.02,1
176,-0.09,-0.08,-0.26,-0.27,-0.26,-0.26,0.02,0.02,-0.02,-0.04,-0.02,0.03,0.00,-0.01,0.07,0.01,0.02,-0.05,-0.05,-0.01,0.03,0.01,0.01,-0.02,-0.04,0.03,0.04,-0.05,0.00,0.04,0.03,-0.02,-0.03,0.00,0.02,-0.01,-0.03,0.00,0.05,0.02,-0.01,-0.02,-0.02,0.03,-0.02,-0.02,0.00,0.00,0.01,0.00,-0.02,0.01,0.02,0.00,-0.02,0.00,0.01,-0.01,-0.02,0.01,0.01,0.01,0.01,-0.04,0.56,0.55,-0.03,-0.03,0.03,-0.24,-0.24,1
177,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,-0.02,-0.01,-0.02,-0.01,0.01,0.01,-0.02,-0.01,0.02,0.03,0.01,0.00,0.00,-0.01,-0.02,-0.01,0.02,0.03,-0.02,-0.02,-0.04,0.00,1.22,0.00,-0.10,-0.43,-0.43,-0.13,-0.12,0.04,0.03,0.07,0.34,0.34,-0.11,-0.11,0.40,0.40,0.21,0.21,-0.36,-0.35,0.01,0.15,0.15,0.07,0.06,0.03,0.03,0.00,0.00,0.01,-0.23,-0.22,-0.01,-0.01,-0.06,0.07,0.07,0.00,-0.01,-0.04,-0.04,2
178,0.11,-0.04,-0.04,0.00,0.04,-0.03,-0.03,0.01,0.04,0.00,0.00,0.01,-0.01,-0.01,0.00,-0.02,0.02,0.02,-0.01,-0.02,0.03,0.01,0.00,-0.05,0.00,0.55,0.55,-0.17,-0.18,-0.33,-0.33,-0.04,0.00,0.00,0.00,-0.04,0.01,0.02,0.01,1.54,1.54,-0.26,-0.27,-0.10,0.27,0.27,-0.39,-0.40,-0.44,-0.45,-0.01,0.00,-0.03,-0.03,0.00,-

In [ ]:
#hot_test=diff_test.loc[diff_test.Class=='Hot',:].drop('Class',axis=1)
#cold_test=diff_test.loc[diff_test.Class=='Cold',:].drop('Class',axis=1)
#print(hot_test)
#print(cold_test)

In [ ]:
#hot_label=pycaret.regression.predict_model(best_hot_reg_model,hot_test)['Label']
#cold_label=pycaret.regression.predict_model(best_cold_reg_model,cold_test)['Label']
#label=pd.concat([hot_label,cold_label])
#label=label.sort_index()
#label

In [ ]:
submit=pd.DataFrame({'Day':range(1,181),'kW-12:00-23:50':pycaret.regression.predict_model(best_reg_model,diff_test)['Label']}).set_index('Day')
submit.to_csv('Submission.csv')

NameError: ignored